In [416]:
import pandas as pd
import plotly.express as px
import re
import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

from Project.DatabaseLocator import DatabaseLocator

Locator = DatabaseLocator()

In [39]:
netzero1 = pd.read_csv(
    filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/All-Subsystems-minute-year1.csv"))

In [40]:
netzero2 = pd.read_csv(
    filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/All-Subsystems-minute-year2.csv"))

In [41]:
netzero1_hourly = pd.read_csv(
    filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/All-Subsystems-hour-year1.csv"))

In [42]:
netzero2_hourly = pd.read_csv(
    filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/All-Subsystems-hour-year2.csv"))

In [43]:
metadata = pd.read_csv(filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/Metadata-minute.csv"))

In [44]:
netzero1["Timestamp"] = pd.to_datetime(netzero1["Timestamp"], format="%Y-%m-%d %H:%M:%S%z")

In [105]:
netzero1_hourly["Timestamp"] = pd.to_datetime(netzero1_hourly["Timestamp"], format="%Y-%m-%d %H:%M:%S%z")

In [45]:
netzero1.loc[:50]

,Timestamp,TimeStamp_Count,DHW_ClothesWasherColdFlow,DHW_ClothesWasherHotFlow,DHW_DishwasherHotFlow,DHW_HeatPumpWaterHeaterEnergyTotal,DHW_HeatPumpWaterHeaterPowerTotal,DHW_ManifoldColdFlow,DHW_ManifoldHotFlow,DHW_MixValveColdFlow,...,Vent_HRVTempReturnIn,Vent_HRVTempSupplyOut,Vent_HRVfreshairpressuredifferential,Vent_HRVstaleairpressuredifferential,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierAirflow,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierPower
0,2013-07-01 00:01:14-04:00,1,0.0,0.0,0.0,0.000,6.92,0.0,0.0,0.0,...,22.552117,23.758415,NaN,NaN,125.4000,922.6568,NaN,NaN,NaN,NaN
1,2013-07-01 00:02:14-04:00,2,0.0,0.0,0.0,0.132,6.92,0.0,0.0,0.0,...,22.523656,23.737489,NaN,NaN,127.0400,919.2424,NaN,NaN,NaN,NaN
2,2013-07-01 00:03:14-04:00,3,0.0,0.0,0.0,0.264,6.92,0.0,0.0,0.0,...,22.539345,23.742217,NaN,NaN,153.3050,920.8184,NaN,NaN,NaN,NaN
3,2013-07-01 00:04:14-04:00,4,0.0,0.0,0.0,0.396,6.92,0.0,0.0,0.0,...,22.539554,23.727526,NaN,NaN,143.4575,922.3944,NaN,NaN,NaN,NaN
4,2013-07-01 00:05:14-04:00,5,0.0,0.0,0.0,0.528,6.92,0.0,0.0,0.0,...,22.506425,23.705258,NaN,NaN,141.8150,925.2832,NaN,NaN,NaN,NaN
5,2013-07-01 00:06:14-04:00,6,0.0,0.0,0.0,0.660,6.92,0.0,0.0,0.0,...,22.491386,23.715523,NaN,NaN,146.7400,932.9000,NaN,NaN,NaN,NaN
6,2013-07-01 00:07:14-04:00,7,0.0,0.0,0.0,0.792,6.92,0.0,0.0,0.0,...,22.526583,23.697305,NaN,NaN,145.9175,928.1728,NaN,NaN,NaN,NaN
7,2013-07-01 00:08:14-04:00,8,0.0,0.0,0.0,0.924,6.92,0.0,0.0,0.0,...,22.469212,23.691279,NaN,NaN,152.4850,950.4968,NaN,NaN,NaN,NaN
8,2013-07-01 00:09:14-04:00,9,0.0,0.0,0.0,1.056,6.92,0.0,0.0,0.0,...,22.466588,23.667564,NaN,NaN,161.5125,956.0128,NaN,NaN,NaN,NaN
9,2013-07-01 00:10:14-04:00,10,0.0,0.0,0.0,1.188,6.92,0.0,0.0,0.0,...,22.453474,23.688705,NaN,NaN,156.5875,946.2952,NaN,NaN,NaN,NaN


In [46]:
netzero2.loc[:50]

,Timestamp,TimeStamp_Count,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,...,SHW_GlycolFlowHXCoriolisSHW,SHW_WaterFlowHXCoriolisSHW,SHW_GlycolFlowRateHXCoriolisSHW,SHW_WaterFlowRateHXCoriolisSHW,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow
0,2015-02-01 00:01:50-05:00,1,0.000000,4.994715,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,184.412,1108.765,4.471,72.102,74.655,0.0
1,2015-02-01 00:02:50-05:00,2,0.000000,5.046792,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,184.618,1097.577,4.508,72.096,74.701,0.0
2,2015-02-01 00:03:50-05:00,3,0.000000,5.129493,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,169.844,1086.113,4.510,72.077,74.759,0.0
3,2015-02-01 00:04:50-05:00,4,0.001981,5.163539,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,167.300,1079.586,4.529,72.074,74.829,0.0
4,2015-02-01 00:05:50-05:00,5,0.001981,5.232744,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,182.976,1048.502,4.438,72.086,74.863,0.0
5,2015-02-01 00:06:50-05:00,6,0.001981,5.349794,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,171.732,1055.987,4.561,72.077,74.965,0.0
6,2015-02-01 00:07:50-05:00,7,0.001981,5.282022,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,169.638,1047.938,4.475,72.099,75.057,0.0
7,2015-02-01 00:08:50-05:00,8,0.004624,5.294622,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,173.578,1046.296,4.498,72.125,75.177,0.0
8,2015-02-01 00:09:50-05:00,9,0.007265,5.273097,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,176.779,1045.797,4.529,72.114,75.232,0.0
9,2015-02-01 00:10:50-05:00,10,0.007265,5.287171,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,173.250,1043.564,4.527,72.117,75.279,0.0


In [47]:
netzero1_hourly.describe().T

,count,mean,std,min,25%,50%,75%,max
DHW_ClothesWasherColdFlow,8673.0,4.188160,9.225338,0.000000,0.000000,0.000000,0.000000,33.064630
DHW_ClothesWasherHotFlow,8673.0,0.779633,6.223181,0.000000,0.000000,0.033025,0.085865,201.696899
DHW_DishwasherHotFlow,8673.0,0.204105,0.566595,0.000000,0.000000,0.000000,0.000000,4.636710
DHW_HeatPumpWaterHeaterEnergyTotal,8673.0,1594.941097,5025.899160,0.000000,281.087596,1093.601048,2132.736588,188169.777407
DHW_HeatPumpWaterHeaterPowerTotal,8673.0,127.994912,243.000776,0.000000,6.920000,6.920000,169.316904,3556.388661
...,...,...,...,...,...,...,...,...
HVAC_HeatPumpOutdoorUnitPower,8632.0,508.134667,449.981229,1.575893,40.016143,495.075013,820.468290,1993.244867
HVAC_DehumidifierAirflow,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HVAC_DehumidifierExitAirTemp,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HVAC_DehumidifierInletAirTemp,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
netzero2.columns.difference(netzero1.columns)

Index(['DHW_WaterTempBAShowerMixed', 'DHW_WaterTempBAShwCold',
       'DHW_WaterTempBAShwHot', 'DHW_WaterTempBASinkCold',
       'DHW_WaterTempBASinkHot', 'DHW_WaterTempBASinkMixed',
       'DHW_WaterTempMBASinkRCold', 'DHW_WaterTempMBASinkRHot',
       'DHW_WaterTempMBASinkRMixed', 'PV_PVSystem1ACEnergyOSEACPV1OS',
       'PV_PVSystem1ACPowerOSPACPV1OS', 'PV_PVSystem2ACEnergyOSEACPV2OS',
       'PV_PVSystem2ACPowerOSPACPV2OS', 'SHW_GlycolFlowHXCoriolisSHW',
       'SHW_GlycolFlowRateHXCoriolisSHW', 'SHW_WaterFlowHXCoriolisSHW',
       'SHW_WaterFlowRateHXCoriolisSHW', 'Vent_HRVDewpointTemperatureExhaust',
       'Vent_HRVDewpointTemperatureOutdoor',
       'Vent_HRVDewpointTemperatureReturn',
       'Vent_HRVDewpointTemperatureSupply', 'Vent_HRVexhaustflowrate',
       'Vent_HRVoutdoorflowrate', 'Vent_HRVreturnflowrate'],
      dtype='object')

In [49]:
netzero1.columns[3]

'DHW_ClothesWasherHotFlow'

In [50]:
netzero1.columns.tolist()

['Timestamp',
 'TimeStamp_Count',
 'DHW_ClothesWasherColdFlow',
 'DHW_ClothesWasherHotFlow',
 'DHW_DishwasherHotFlow',
 'DHW_HeatPumpWaterHeaterEnergyTotal',
 'DHW_HeatPumpWaterHeaterPowerTotal',
 'DHW_ManifoldColdFlow',
 'DHW_ManifoldHotFlow',
 'DHW_MixValveColdFlow',
 'DHW_RoomTempBasementHPWH',
 'DHW_SHWGlycolTempIn',
 'DHW_SHWGlycolTempOut',
 'DHW_SHWPumpsEnergywithStandby',
 'DHW_SHWPumpsPowerWithStandby',
 'DHW_StatusSolenoidColdKitchenSink',
 'DHW_StatusSolenoidColdMBAShower',
 'DHW_StatusSolenoidColdMBATub',
 'DHW_StatusSolenoidHotKitchenSink',
 'DHW_StatusSolenoidHotMBAShower',
 'DHW_StatusSolenoidHotMBATub',
 'DHW_WaterTempDishwasher',
 'DHW_WaterTempHPWHIn',
 'DHW_WaterTempHPWHOut',
 'DHW_WaterTempKSinkCold',
 'DHW_WaterTempKSinkHot',
 'DHW_WaterTempKSinkMix',
 'DHW_WaterTempMBAShwCold',
 'DHW_WaterTempMBAShwHot',
 'DHW_WaterTempMBAShwMix',
 'DHW_WaterTempMBATubCold',
 'DHW_WaterTempMBATubHot',
 'DHW_WaterTempMBATubMix',
 'DHW_WaterTempManifoldColdIn',
 'DHW_WaterTempManifol

In [51]:
totals = [x for x in netzero1.columns if "Total" in x]
totals

['DHW_HeatPumpWaterHeaterEnergyTotal',
 'DHW_HeatPumpWaterHeaterPowerTotal',
 'Load_DryerEnergyTotal',
 'Load_DryerPowerTotal',
 'Load_OvenEnergyTotal',
 'Load_OvenPowerTotal',
 'PV_PowerFactor3PhTotalPF3PhT1',
 'PV_PowerFactor3PhTotalPF3PhT2',
 'PV_Watts3PhTotalW3PhT1',
 'PV_Watts3PhTotalW3PhT2']

In [ ]:
fig = px.scatter(netzero1, x="Timestamp", y="Load_KPlugLoadsPowerUsage", color="Load_StatusPlugLoadToaster")
fig.show()

In [63]:
plugs = metadata.loc[(metadata['Parameter'] == "Status_OnOff") & (metadata["Measurement_Location"] == "Kitchen") & (
    metadata["Unnamed: 0"].str.contains("Plug"))]["Unnamed: 0"].tolist()  #

In [64]:
kitchenplugs = netzero1_hourly[["Timestamp", "Load_KPlugLoadsPowerUsage"] + plugs].copy()
kitchenplugs["Status"] = kitchenplugs[plugs].sum(axis=1)

In [65]:
(kitchenplugs[plugs] < 0) | (kitchenplugs[plugs] > 1)

,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
8756,False,False,False,False,False,False,False
8757,False,False,False,False,False,False,False
8758,False,False,False,False,False,False,False
8759,False,False,False,False,False,False,False


In [66]:
appliance_dict = {}
for appliance in plugs:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load")[-1]))
    appliance_switch = kitchenplugs[(kitchenplugs[appliance] != kitchenplugs[appliance].shift(1))][
                           ["Timestamp", appliance]][1:]
    appliance_dict[name] = pd.DataFrame(
        {"Timestamp": [appliance_switch.loc[index, "Timestamp"] for index in appliance_switch.index],
         "Appliance": name,
         "Color": ["green" if appliance_switch.loc[index, appliance] == 1 else "red" for index in
                   appliance_switch.index]})

appliance_dict

{'Blender':                      Timestamp Appliance Color
 0    2013-07-01 06:00:00-04:00   Blender   red
 1    2013-07-01 07:00:00-04:00   Blender   red
 2    2013-07-02 06:00:00-04:00   Blender   red
 3    2013-07-02 07:00:00-04:00   Blender   red
 4    2013-07-03 06:00:00-04:00   Blender   red
 ..                         ...       ...   ...
 815  2014-06-28 10:00:00-04:00   Blender   red
 816  2014-06-29 09:00:00-04:00   Blender   red
 817  2014-06-29 10:00:00-04:00   Blender   red
 818  2014-06-30 06:00:00-04:00   Blender   red
 819  2014-06-30 07:00:00-04:00   Blender   red
 
 [820 rows x 3 columns],
 'Toaster Oven':                      Timestamp     Appliance Color
 0    2013-07-01 06:00:00-04:00  Toaster Oven   red
 1    2013-07-01 07:00:00-04:00  Toaster Oven   red
 2    2013-07-02 06:00:00-04:00  Toaster Oven   red
 3    2013-07-02 07:00:00-04:00  Toaster Oven   red
 4    2013-07-03 06:00:00-04:00  Toaster Oven   red
 ..                         ...           ...   ...
 821  

In [68]:
fig = px.scatter(kitchenplugs, x=kitchenplugs["Timestamp"], y="Load_KPlugLoadsPowerUsage", color="Status")

height = 1
for appliance, df in appliance_dict.items():
    for index in df.index:
        x = df.loc[index, "Timestamp"].timestamp() * 1000 - (2 + int(str(df.loc[index, "Timestamp"]).split("-")[-1][
                                                                         1])) * 3_600_000  #Plotly bug work around. Computes unix time with offset
        fig.add_vline(x=x,
                      line_width=3,
                      line_dash="dash",
                      annotation_text=df.loc[index, "Appliance"],
                      line_color=df.loc[index, "Color"],
                      annotation_y=height)
    height -= 0.05

fig.update_layout(
    xaxis={'type': 'date', 'tickformat': '%Y-%m-%d %H:%M:%S'}
)

fig.update_layout(
    xaxis_tickformatstops=[
        dict(dtickrange=[None, 60000], value="%H:%M:%S\n%d %b %y"),
        dict(dtickrange=[60000, 3600000], value="%H:%M\n%d %b %y"),
        dict(dtickrange=[3600000, 86400000], value="%d %b %Y")
    ]
)

fig.show()

KeyboardInterrupt: 

In [70]:
kitchenplugs[kitchenplugs["Load_StatusPlugLoadToasterOven"] == 1][["Timestamp", "Load_StatusPlugLoadToasterOven"]]

,Timestamp,Load_StatusPlugLoadToasterOven


In [ ]:
fig = px.scatter(netzero1.loc[:10000], x="Timestamp", y="Load_DryerPowerTotal")
fig.show()

In [72]:
production = metadata.loc[(metadata['Parameter'] == "Power_Electrical") & (
    metadata["Description"].str.contains("produced"))]["Unnamed: 0"].tolist()

consumption =
metadata.loc[((metadata['Parameter'] == "Power_Electrical") | (metadata['Parameter'] == "Power_Thermal")) & (
    metadata["Description"].str.contains("power consumption" or "used"))]["Unnamed: 0"].tolist()

In [73]:
total_instant_energy_produced1 = netzero1_hourly[production].sum(axis=1)

total_instant_energy_consumed1 = netzero1_hourly[consumption].sum(axis=1)

instant_power_balance1 = total_instant_energy_produced1 - total_instant_energy_consumed1

instant_power_balance_cummulative1 = instant_power_balance1.cumsum()

total_instant_energy_produced2 = netzero2_hourly[production].sum(axis=1)

total_instant_energy_consumed2 = netzero2_hourly[consumption].sum(axis=1)

instant_power_balance2 = total_instant_energy_produced2 - total_instant_energy_consumed2

instant_power_balance_cummulative2 = instant_power_balance2.cumsum()

In [ ]:
fig = px.scatter(netzero1_hourly, x="Timestamp", y=total_instant_energy_produced1.cumsum())
fig.show()

In [ ]:
fig = px.scatter(netzero1_hourly, x="Timestamp", y=total_instant_energy_consumed1.cumsum())
fig.show()

In [ ]:
fig = px.scatter(netzero1_hourly, x="Timestamp", y=instant_power_balance1)
fig.show()

In [390]:
fig = px.line(netzero1_hourly, x="Timestamp", y=instant_power_balance_cummulative1)
fig.show()

In [ ]:
fig = px.line(netzero2_hourly, x="Timestamp", y=instant_power_balance_cummulative2)
fig.show()

In [81]:
appliance_instant_energy_consumed1 = netzero1_hourly[consumption].sum(axis=0).sort_values(ascending=False)
appliance_instant_energy_consumed1

HVAC_HeatPumpOutdoorUnitPower        4.386218e+06
HVAC_HeatPumpIndoorUnitPower         1.780827e+06
DHW_HeatPumpWaterHeaterPowerTotal    1.110100e+06
Load_2ndFloorSensHeatPowerUsage      1.073024e+06
Load_1stFloorSensHeatPowerUsage      5.665715e+05
                                         ...     
Elec_PowerPlugsInstHall              0.000000e+00
Elec_PowerPlugsBA2South              0.000000e+00
Elec_PowerPlugsInstLRB               0.000000e+00
Elec_PowerPlugsInstMudroomB          0.000000e+00
HVAC_DehumidifierPower               0.000000e+00
Length: 78, dtype: float64

In [82]:
fig = px.bar(appliance_instant_energy_consumed1, x=appliance_instant_energy_consumed1)
fig.show()

In [423]:
kitchenplugs = netzero1_hourly[["Timestamp"] + plugs].copy()
kitchenplugs["Status"] = kitchenplugs[plugs].sum(axis=1)
kitchenplugs

,Timestamp,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker,Status
0,2013-07-01 00:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-07-01 01:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-07-01 02:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-07-01 03:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-07-01 04:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
8756,2014-06-30 20:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8757,2014-06-30 21:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8758,2014-06-30 22:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8759,2014-06-30 23:00:00-04:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [424]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
kitchenplugs['Timestamp'] = pd.to_datetime(kitchenplugs['Timestamp'], errors="coerce", utc=True, format="%Y-%m-%d %H:%M:%S%z")

for index in kitchenplugs.index:
    kitchenplugs.loc[index, "Timestamp"] = kitchenplugs.loc[index, "Timestamp"] - datetime.timedelta(seconds=(int(str(netzero1_hourly.loc[index, "Timestamp"]).split("-")[-1][1])) * 3_600)


#Extracts hour of Timestamp.
kitchenplugs["HourOfTimestamp"] = kitchenplugs.Timestamp.dt.hour

#Replaces values to be binary 0 or 1.
for i in plugs:
    kitchenplugs.loc[kitchenplugs[i] < 0, i] = 0
    kitchenplugs.loc[kitchenplugs[i] > 0.0001, i] = 1

kitchenplugs

,Timestamp,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker,Status,HourOfTimestamp
0,2013-07-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2013-07-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,2013-07-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2013-07-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,2013-07-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...
8756,2014-06-30 20:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
8757,2014-06-30 21:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21
8758,2014-06-30 22:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22
8759,2014-06-30 23:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23


In [425]:
#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for a selected attribute in kitchenplugs
count_series = kitchenplugs.groupby(["HourOfTimestamp", "Load_StatusPlugLoadHandMixer"]).size()

new_df = count_series.to_frame(name='size').reset_index()

only_on = new_df.loc[new_df["Load_StatusPlugLoadHandMixer"] == 1]

In [426]:
#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for all attributes in kitchenplugs

dataframes = list()
for i in plugs:
    count_series = kitchenplugs.groupby(["HourOfTimestamp", i]).size()

    new_df = count_series.to_frame(name=f'{i}_ON').reset_index()

    only_on = new_df.loc[new_df[i] == 1].copy()
    only_on.drop(columns=[i], inplace=True)
    dataframes.append(only_on)

hourON = pd.DataFrame(list(range(0, 24)), columns=["HourOfTimestamp"])
for i in dataframes:
    hourON = hourON.merge(i, how="outer", on="HourOfTimestamp").fillna(0).sort_values(by='HourOfTimestamp',
                                                                                      ascending=True)

hourON

,HourOfTimestamp,Load_StatusPlugLoadBlender_ON,Load_StatusPlugLoadToasterOven_ON,Load_StatusPlugLoadSlowCooker_ON,Load_StatusPlugLoadToaster_ON,Load_StatusPlugLoadHandMixer_ON,Load_StatusPlugLoadCanOpener_ON,Load_StatusPlugLoadCoffeeMaker_ON
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,2.0,2.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,256.0,256.0,0.0,256.0,0.0,0.0,256.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,13.0
8,8,0.0,0.0,52.0,0.0,0.0,0.0,0.0
9,9,101.0,101.0,102.0,101.0,0.0,0.0,101.0


In [427]:
def normalise_dataframe(df):
    '''
    Normalises values of all attributes in a dataframe, with minmax method.
    :param df:
    :type df:
    :return:
    :rtype:
    '''
    normalised_df = df.copy()

    for column in normalised_df:
        try:
            normalised_df[column] = (normalised_df[column] - normalised_df[column].min()) / (
                    normalised_df[column].max() - normalised_df[column].min())
        except:
            normalised_df[column] = normalised_df[column]

    return normalised_df

In [428]:
import plotly.graph_objects as go

#set up data to heatmap
columns = hourON.columns[1:].tolist()
hourON[columns] = normalise_dataframe(hourON[columns])
z_data = []
for i in hourON[columns]:
    data_list = hourON[i].round(2).values.tolist()
    z_data.append(data_list)

#Annotate z_values
annotations = go.Annotations()
for n, row in enumerate(z_data):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text="" if z_data[n][m] == 0 else str(z_data[n][m]),
                                         x=hourON["HourOfTimestamp"][m], y=columns[n], xref='x1', yref='y1',
                                         showarrow=False))

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=hourON["HourOfTimestamp"],
    y=columns,
    colorscale='Teal')
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30, annotations=annotations, )

fig.update_xaxes(title="Hour of day")
fig.update_yaxes(title="Appliance", showticklabels=True)
fig.show()

/usr/local/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:62: DeprecationWarning:

plotly.graph_objs.Annotations is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


/usr/local/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:143: DeprecationWarning:

plotly.graph_objs.Annotation is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


